In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [2]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, BertModel, get_linear_schedule_with_warmup, TrainingArguments, Trainer, EvalPrediction
import torch.nn as nn
from torch.optim import AdamW
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os
from tqdm import tqdm

In [3]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the datasets
print("Loading datasets...")
sst2 = load_dataset("glue", "sst2")
imdb = load_dataset("imdb")

# Load the pre-trained BERT model fine-tuned on IMDB
model_name = "yyammerrrss/imdb-sft-bert"
print(f"Loading model from {model_name}...")
tokenizer = BertTokenizer.from_pretrained(model_name)

Using device: cuda
Loading datasets...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

...

Loading model from yyammerrrss/imdb-sft-bert...


tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

In [4]:
# Define the tokenization functions
def tokenize_sst2(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

def tokenize_imdb(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Tokenize the datasets
print("Tokenizing datasets...")
tokenized_sst2 = {}
for split in sst2:
    tokenized_sst2[split] = sst2[split].map(tokenize_sst2, batched=True)
    tokenized_sst2[split].set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# For IMDB, we'll use both train and test sets
tokenized_imdb = {}
tokenized_imdb['train'] = imdb['train'].map(tokenize_imdb, batched=True)
tokenized_imdb['test'] = imdb['test'].map(tokenize_imdb, batched=True)
for split in tokenized_imdb:
    tokenized_imdb[split].set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

print(f"SST2 train size: {len(tokenized_sst2['train'])}")
print(f"SST2 validation size: {len(tokenized_sst2['validation'])}")
print(f"IMDB train size: {len(tokenized_imdb['train'])}")
print(f"IMDB test size: {len(tokenized_imdb['test'])}")

Tokenizing datasets...


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

SST2 train size: 67349
SST2 validation size: 872
IMDB train size: 25000
IMDB test size: 25000


In [5]:
# Define the adversarial model architecture
class AdversarialBert(nn.Module):
    def __init__(self, bert_model_name, num_labels=2, lambda_param=0.1):
        super(AdversarialBert, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.domain_classifier = nn.Linear(self.bert.config.hidden_size, 2)
        self.lambda_param = lambda_param

    def forward(self, input_ids, attention_mask, labels=None, domain_labels=None, alpha=1.0):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        # Task classifier (sentiment)
        logits = self.classifier(pooled_output)

        # Domain classifier with gradient reversal
        if self.training and domain_labels is not None:
            # Apply gradient reversal
            reverse_feature = GradientReversalFunction.apply(pooled_output, alpha)
            domain_logits = self.domain_classifier(reverse_feature)

            # Calculate losses
            task_loss = F.cross_entropy(logits, labels)
            domain_loss = F.cross_entropy(domain_logits, domain_labels)
            loss = task_loss + self.lambda_param * domain_loss

            return {
                'loss': loss,
                'task_logits': logits,
                'domain_logits': domain_logits,
                'task_loss': task_loss,
                'domain_loss': domain_loss
            }
        else:
            # During evaluation, only use the task classifier
            loss = F.cross_entropy(logits, labels) if labels is not None else None
            return {
                'loss': loss,
                'task_logits': logits
            }

# Define gradient reversal layer for adversarial training
class GradientReversalFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.clone()

    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.alpha * grad_output, None

# Function to create dataloaders for adversarial training
def create_adversarial_dataloader(source_dataset, target_dataset, batch_size):
    # Create domain labels: 0 for source domain (IMDB), 1 for target domain (SST2)
    source_domain_labels = torch.zeros(len(source_dataset), dtype=torch.long)
    target_domain_labels = torch.ones(len(target_dataset), dtype=torch.long)

    # Sample from the source dataset to match the size of the target dataset
    source_indices = np.random.choice(len(source_dataset), min(len(source_dataset), len(target_dataset)), replace=False)
    source_indices = [int(idx) for idx in source_indices]  # Convert numpy.int64 to Python int

    # For IMDB (source), truncate the tensors to match SST2's length (128)
    source_data = {
        'input_ids': torch.stack([source_dataset[i]['input_ids'][:128] for i in source_indices]),
        'attention_mask': torch.stack([source_dataset[i]['attention_mask'][:128] for i in source_indices]),
        'labels': torch.stack([source_dataset[i]['label'] for i in source_indices]),
        'domain_labels': source_domain_labels[source_indices]
    }

    # Get all target data (SST2)
    target_data = {
        'input_ids': torch.stack([target_dataset[i]['input_ids'] for i in range(len(target_dataset))]),
        'attention_mask': torch.stack([target_dataset[i]['attention_mask'] for i in range(len(target_dataset))]),
        'labels': torch.stack([target_dataset[i]['label'] for i in range(len(target_dataset))]),
        'domain_labels': target_domain_labels
    }

    # Combine source and target data
    combined_input_ids = torch.cat([source_data['input_ids'], target_data['input_ids']], dim=0)
    combined_attention_mask = torch.cat([source_data['attention_mask'], target_data['attention_mask']], dim=0)
    combined_labels = torch.cat([source_data['labels'], target_data['labels']], dim=0)
    combined_domain_labels = torch.cat([source_data['domain_labels'], target_domain_labels], dim=0)

    # Create dataset and dataloader
    dataset = TensorDataset(combined_input_ids, combined_attention_mask, combined_labels, combined_domain_labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return dataloader

In [6]:
# Define evaluation function
def evaluate_model(model, eval_dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in eval_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs['task_logits']

            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Create evaluation dataloaders
def create_eval_dataloader(dataset, batch_size):
    # Check if this is the IMDB dataset (which has length 512)
    first_input_ids = dataset[0]['input_ids']

    if len(first_input_ids) > 128:
        input_ids = torch.stack([dataset[i]['input_ids'][:128] for i in range(len(dataset))])
        attention_mask = torch.stack([dataset[i]['attention_mask'][:128] for i in range(len(dataset))])
    else:
        input_ids = torch.stack([dataset[i]['input_ids'] for i in range(len(dataset))])
        attention_mask = torch.stack([dataset[i]['attention_mask'] for i in range(len(dataset))])

    labels = torch.stack([dataset[i]['label'] for i in range(len(dataset))])

    eval_dataset = TensorDataset(input_ids, attention_mask, labels)
    eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size)

    return eval_dataloader

In [7]:
# Main training function
def train_adversarial(source_train, target_train, target_val, target_test, lambda_param=0.1,
                      batch_size=16, num_epochs=3, learning_rate=2e-5, weight_decay=0.01):
    # Initialize model
    model = AdversarialBert(model_name, lambda_param=lambda_param)
    model.to(device)

    # Create dataloaders
    train_dataloader = create_adversarial_dataloader(source_train, target_train, batch_size)
    target_val_dataloader = create_eval_dataloader(target_val, batch_size)
    target_test_dataloader = create_eval_dataloader(target_test, batch_size)

    # Initialize optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    total_steps = len(train_dataloader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # Training loop
    print("Starting adversarial training...")
    best_val_accuracy = 0
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        epoch_task_loss = 0
        epoch_domain_loss = 0

        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for batch in progress_bar:
            input_ids, attention_mask, labels, domain_labels = [b.to(device) for b in batch]

            # Clear gradients
            optimizer.zero_grad()

            # Calculate p value for increasing domain influence over time
            p = float(epoch) / num_epochs
            alpha = 2. / (1. + np.exp(-10 * p)) - 1

            # Forward pass
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
                domain_labels=domain_labels,
                alpha=alpha
            )

            # Backward pass
            outputs['loss'].backward()
            optimizer.step()
            scheduler.step()

            # Update progress bar
            epoch_loss += outputs['loss'].item()
            epoch_task_loss += outputs['task_loss'].item()
            epoch_domain_loss += outputs['domain_loss'].item()
            progress_bar.set_postfix({
                'loss': epoch_loss / (progress_bar.n + 1),
                'task_loss': epoch_task_loss / (progress_bar.n + 1),
                'domain_loss': epoch_domain_loss / (progress_bar.n + 1)
            })

        # Evaluate on target validation set
        print(f"Evaluating on target validation set (epoch {epoch+1})...")
        val_metrics = evaluate_model(model, target_val_dataloader)
        print(f"Validation metrics: {val_metrics}")

        # Save best model
        if val_metrics['accuracy'] > best_val_accuracy:
            best_val_accuracy = val_metrics['accuracy']
            print(f"New best model with validation accuracy: {best_val_accuracy:.4f}")
            output_dir = "./results/adversarial_finetuned"
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            torch.save(model.state_dict(), os.path.join(output_dir, "model.bin"))

    # Load best model for final evaluation
    print("Loading best model for final evaluation...")
    model.load_state_dict(torch.load(os.path.join(output_dir, "model.bin")))

    # Evaluate on target validation and test sets
    print("Final evaluation on target validation set...")
    final_val_metrics = evaluate_model(model, target_val_dataloader)
    print(f"Final validation metrics: {final_val_metrics}")

    print("Evaluating on target test set...")
    final_test_metrics = evaluate_model(model, target_test_dataloader)
    print(f"Target test metrics: {final_test_metrics}")

    # Evaluate on source test set (IMDB)
    print("Evaluating on source test set (IMDB)...")
    source_test_dataloader = create_eval_dataloader(tokenized_imdb['test'], batch_size)
    source_test_metrics = evaluate_model(model, source_test_dataloader)
    print(f"Source test metrics: {source_test_metrics}")

    return {
        'model': model,
        'target_val_metrics': final_val_metrics,
        'target_test_metrics': final_test_metrics,
        'source_test_metrics': source_test_metrics
    }

# Run the adversarial training
print("Starting adversarial finetuning experiment...")
results = train_adversarial(
    source_train=tokenized_imdb['train'],
    target_train=tokenized_sst2['train'],
    target_val=tokenized_sst2['validation'],
    target_test=tokenized_sst2['validation'],
    batch_size=16,
    num_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01
)

Starting adversarial finetuning experiment...


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Starting adversarial training...


Epoch 1/3: 100%|██████████| 5772/5772 [07:55<00:00, 12.14it/s, loss=0.243, task_loss=0.203, domain_loss=0.398]


Evaluating on target validation set (epoch 1)...
Validation metrics: {'accuracy': 0.926605504587156, 'precision': 0.9377880184331797, 'recall': 0.9166666666666666, 'f1': 0.9271070615034168}
New best model with validation accuracy: 0.9266


Epoch 2/3: 100%|██████████| 5772/5772 [07:54<00:00, 12.16it/s, loss=0.259, task_loss=0.114, domain_loss=1.44]


Evaluating on target validation set (epoch 2)...
Validation metrics: {'accuracy': 0.9231651376146789, 'precision': 0.929384965831435, 'recall': 0.918918918918919, 'f1': 0.9241223103057757}


Epoch 3/3: 100%|██████████| 5772/5772 [07:54<00:00, 12.16it/s, loss=0.125, task_loss=0.058, domain_loss=0.665]


Evaluating on target validation set (epoch 3)...
Validation metrics: {'accuracy': 0.9243119266055045, 'precision': 0.9108695652173913, 'recall': 0.9436936936936937, 'f1': 0.9269911504424779}
Loading best model for final evaluation...
Final evaluation on target validation set...
Final validation metrics: {'accuracy': 0.926605504587156, 'precision': 0.9377880184331797, 'recall': 0.9166666666666666, 'f1': 0.9271070615034168}
Evaluating on target test set...
Target test metrics: {'accuracy': 0.926605504587156, 'precision': 0.9377880184331797, 'recall': 0.9166666666666666, 'f1': 0.9271070615034168}
Evaluating on source test set (IMDB)...
Source test metrics: {'accuracy': 0.89308, 'precision': 0.8874694424729911, 'recall': 0.90032, 'f1': 0.8938485365950518}


In [8]:
# Display final results
print("\n\n" + "="*80)
print("ADVERSARIAL DOMAIN ADAPTATION RESULTS")
print("="*80)

print(f"Target validation (SST2) results:")
print(f"Accuracy: {results['target_val_metrics']['accuracy']:.4f}")
print(f"F1 Score: {results['target_val_metrics']['f1']:.4f}")
print(f"Precision: {results['target_val_metrics']['precision']:.4f}")
print(f"Recall: {results['target_val_metrics']['recall']:.4f}")

print(f"\nSource test (IMDB) results:")
print(f"Accuracy: {results['source_test_metrics']['accuracy']:.4f}")
print(f"F1 Score: {results['source_test_metrics']['f1']:.4f}")
print(f"Precision: {results['source_test_metrics']['precision']:.4f}")
print(f"Recall: {results['source_test_metrics']['recall']:.4f}")




ADVERSARIAL DOMAIN ADAPTATION RESULTS
Target validation (SST2) results:
Accuracy: 0.9266
F1 Score: 0.9271
Precision: 0.9378
Recall: 0.9167

Source test (IMDB) results:
Accuracy: 0.8931
F1 Score: 0.8938
Precision: 0.8875
Recall: 0.9003


COMPARISON WITH OTHER DOMAIN ADAPTATION APPROACHES
Approach                  Accuracy   F1         Precision  Recall    
-----------------------------------------------------------------
adversarial               0.9266     0.9271     0.9378     0.9167    
